In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn import neighbors
from sklearn import metrics
from sklearn import model_selection
from sklearn import tree
from sklearn import preprocessing
from sklearn.decomposition import PCA

#Need to put the path to your diann output matrix in here. I renamed the files in notepad quickly before this step as the
#names were very long. Also added empty potential contaminant and reverse columns so that it would work in Proteus later.
df = pd.read_csv("C:/Users/ASUS/Desktop/diann-output.pg_matrix_cleaned_outliers_removed.txt", sep="\t")
df.head()

,Majority protein IDs,Protein.Ids,Protein.Names,Genes,First.Protein.Description,Intensity 85,Intensity QC_G10,Intensity QC_A3,Intensity 75,Intensity 71,...,Intensity 127,Intensity 31,Intensity 1,Intensity 115,Intensity 35,Intensity 91,Intensity 121,Intensity 107,Potential contaminant,Reverse
0,A0A075B6H7,A0A075B6H7,NaN,IGKV3-7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,780180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0A075B6K4,A0A075B6K4,NaN,IGLV3-10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,306602.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A0A075B6S2,A0A075B6S2,NaN,IGKV2D-29,NaN,1471.09,NaN,NaN,NaN,NaN,...,NaN,NaN,25570.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A0A075B6S5,A0A075B6S5,NaN,IGKV1-27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,36935.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A0A075B7D0,A0A075B7D0,NaN,IGHV1OR15-1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,142458.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Categorical Annotation

Using table manipulation and for loops to assign a new heading that specifies whether a sample is from a case or control or QC cohort.

In [3]:
df = df.transpose()
df

,0,1,2,3,4,5,6,7,8,9,...,670,671,672,673,674,675,676,677,678,679
Majority protein IDs,A0A075B6H7,A0A075B6K4,A0A075B6S2,A0A075B6S5,A0A075B7D0,A0A087WSY6,A0A0A0MS15,A0A0B4J1Y8,A0A0B4J1Y9,A0A0C4DH24,...,Q9UQ80,Q9Y287,Q9Y2G1,Q9Y490,Q9Y4L1,Q9Y5C1,Q9Y5Y7,Q9Y646,Q9Y6N7,Q9Y6Z7
Protein.Ids,A0A075B6H7,A0A075B6K4,A0A075B6S2,A0A075B6S5,A0A075B7D0,A0A087WSY6,A0A0A0MS15,A0A0B4J1Y8,A0A0B4J1Y9,A0A0C4DH24,...,Q9UQ80,Q9Y287,Q9Y2G1,Q9Y490,Q9Y4L1,Q9Y5C1,Q9Y5Y7,Q9Y646,Q9Y6N7,Q9Y6Z7
Protein.Names,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Genes,IGKV3-7,IGLV3-10,IGKV2D-29,IGKV1-27,IGHV1OR15-1,IGKV3D-15,IGHV3-49,IGLV9-49,IGHV3-72,IGKV6-21,...,PA2G4,ITM2B,MYRF,TLN1,HYOU1,ANGPTL3,LYVE1,CPQ,ROBO1,COLEC10
First.Protein.Description,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Intensity 91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,551.679,...,5793.53,4894.96,7970.31,12984.0,11005.1,3964.51,107999.0,12017.5,2069.04,24553.1
Intensity 121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,14452.4,19011.5,10740.1,2533.34,68237.1,15823.9,3888.02,28674.6
Intensity 107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,25034.9,10229.5,NaN,44161.4,20288.9,NaN,22333.6
Potential contaminant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.insert(0, "Label", " ")
df

,Label,0,1,2,3,4,5,6,7,8,...,670,671,672,673,674,675,676,677,678,679
Majority protein IDs,,A0A075B6H7,A0A075B6K4,A0A075B6S2,A0A075B6S5,A0A075B7D0,A0A087WSY6,A0A0A0MS15,A0A0B4J1Y8,A0A0B4J1Y9,...,Q9UQ80,Q9Y287,Q9Y2G1,Q9Y490,Q9Y4L1,Q9Y5C1,Q9Y5Y7,Q9Y646,Q9Y6N7,Q9Y6Z7
Protein.Ids,,A0A075B6H7,A0A075B6K4,A0A075B6S2,A0A075B6S5,A0A075B7D0,A0A087WSY6,A0A0A0MS15,A0A0B4J1Y8,A0A0B4J1Y9,...,Q9UQ80,Q9Y287,Q9Y2G1,Q9Y490,Q9Y4L1,Q9Y5C1,Q9Y5Y7,Q9Y646,Q9Y6N7,Q9Y6Z7
Protein.Names,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Genes,,IGKV3-7,IGLV3-10,IGKV2D-29,IGKV1-27,IGHV1OR15-1,IGKV3D-15,IGHV3-49,IGLV9-49,IGHV3-72,...,PA2G4,ITM2B,MYRF,TLN1,HYOU1,ANGPTL3,LYVE1,CPQ,ROBO1,COLEC10
First.Protein.Description,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Intensity 91,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5793.53,4894.96,7970.31,12984.0,11005.1,3964.51,107999.0,12017.5,2069.04,24553.1
Intensity 121,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,14452.4,19011.5,10740.1,2533.34,68237.1,15823.9,3888.02,28674.6
Intensity 107,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,25034.9,10229.5,NaN,44161.4,20288.9,NaN,22333.6
Potential contaminant,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = df.transpose()
df

,Majority protein IDs,Protein.Ids,Protein.Names,Genes,First.Protein.Description,Intensity 85,Intensity QC_G10,Intensity QC_A3,Intensity 75,Intensity 71,...,Intensity 127,Intensity 31,Intensity 1,Intensity 115,Intensity 35,Intensity 91,Intensity 121,Intensity 107,Potential contaminant,Reverse
Label,,,,,,,,,,,...,,,,,,,,,,
0,A0A075B6H7,A0A075B6H7,NaN,IGKV3-7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,780180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0A075B6K4,A0A075B6K4,NaN,IGLV3-10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,306602.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A0A075B6S2,A0A075B6S2,NaN,IGKV2D-29,NaN,1471.09,NaN,NaN,NaN,NaN,...,NaN,NaN,25570.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A0A075B6S5,A0A075B6S5,NaN,IGKV1-27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,36935.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,Q9Y5C1,Q9Y5C1,NaN,ANGPTL3,NaN,5577.84,4138.06,4011.38,5248.27,3342.73,...,4269.59,3281.33,NaN,4102.91,2963.73,3964.51,2533.34,NaN,NaN,NaN
676,Q9Y5Y7,Q9Y5Y7,NaN,LYVE1,NaN,26639.6,37457.2,49709.4,30120.5,36232.6,...,66596.0,70511.9,68331.6,59306.4,48793.4,107999.0,68237.1,44161.4,NaN,NaN
677,Q9Y646,Q9Y646,NaN,CPQ,NaN,12229.7,9045.77,9757.3,14903.9,13347.3,...,9094.25,9484.76,15927.0,10066.5,14565.6,12017.5,15823.9,20288.9,NaN,NaN
678,Q9Y6N7,Q9Y6N7,NaN,ROBO1,NaN,4208.16,2697.48,4508.26,5846.31,4531.03,...,4135.06,9523.76,NaN,4705.79,7478.49,2069.04,3888.02,NaN,NaN,NaN


In [6]:
groupsdf = pd.read_csv("C:/Users/ASUS/Desktop/meta_DIA_jup.csv")
groupsdf.head()

,name,condition
0,Intensity 1,Autism
1,Intensity 10,Autism
2,Intensity 101,Autism
3,Intensity 107,Autism
4,Intensity 113,Autism


In [7]:
groupsdf = groupsdf.set_index("name").transpose()
groupsdf.head()

name,Intensity 1,Intensity 10,Intensity 101,Intensity 107,Intensity 113,Intensity 119,Intensity 131,Intensity 139,Intensity 19,Intensity 29,...,Intensity 4,Intensity 7,Intensity QC_A3,Intensity QC_C3,Intensity QC_B3,Intensity QC_D3,Intensity QC_E3,Intensity QC_F3,Intensity QC_G3,Intensity QC_G10
condition,Autism,Autism,Autism,Autism,Autism,Autism,Autism,Autism,Autism,Autism,...,Neurotypical,Neurotypical,QC,QC,QC,QC,QC,QC,QC,QC


In [8]:
for col1 in df:
    for col2 in groupsdf:
        if col1 == col2:
            df.loc["Label", col1] = groupsdf.loc["condition", col2]

In [11]:
for col in df: print(col)

Majority protein IDs
Protein.Ids
Protein.Names
Genes
First.Protein.Description
Intensity 85
Intensity QC_G10
Intensity QC_A3
Intensity 75
Intensity 71
Intensity 25
Intensity 73
Intensity 53
Intensity 63
Intensity 19
Intensity 65
Intensity 27
Intensity 7
Intensity 135
Intensity 67
Intensity 97
Intensity 101
Intensity 51
Intensity 117
Intensity 16
Intensity 13
Intensity 10
Intensity 109
Intensity 59
Intensity 113
Intensity 105
Intensity 69
Intensity 29
Intensity 81
Intensity 55
Intensity QC_G3
Intensity 89
Intensity 47
Intensity 119
Intensity 87
Intensity 39
Intensity 43
Intensity 129
Intensity QC_B3
Intensity QC_F3
Intensity 123
Intensity 45
Intensity 137
Intensity QC_D3
Intensity 61
Intensity 131
Intensity QC_E3
Intensity 79
Intensity 83
Intensity 4
Intensity QC_C3
Intensity 49
Intensity 37
Intensity 139
Intensity 111
Intensity 57
Intensity 41
Intensity 77
Intensity 133
Intensity 127
Intensity 31
Intensity 1
Intensity 115
Intensity 35
Intensity 91
Intensity 121
Intensity 107
Potential 

In [12]:
df

,Majority protein IDs,Protein.Ids,Protein.Names,Genes,First.Protein.Description,Intensity 85,Intensity QC_G10,Intensity QC_A3,Intensity 75,Intensity 71,...,Intensity 127,Intensity 31,Intensity 1,Intensity 115,Intensity 35,Intensity 91,Intensity 121,Intensity 107,Potential contaminant,Reverse
Label,,,,,,Neurotypical,QC,QC,Neurotypical,Autism,...,Neurotypical,Neurotypical,Autism,Neurotypical,Autism,Neurotypical,Neurotypical,Autism,,
0,A0A075B6H7,A0A075B6H7,NaN,IGKV3-7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,780180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0A075B6K4,A0A075B6K4,NaN,IGLV3-10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,306602.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A0A075B6S2,A0A075B6S2,NaN,IGKV2D-29,NaN,1471.09,NaN,NaN,NaN,NaN,...,NaN,NaN,25570.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A0A075B6S5,A0A075B6S5,NaN,IGKV1-27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,36935.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,Q9Y5C1,Q9Y5C1,NaN,ANGPTL3,NaN,5577.84,4138.06,4011.38,5248.27,3342.73,...,4269.59,3281.33,NaN,4102.91,2963.73,3964.51,2533.34,NaN,NaN,NaN
676,Q9Y5Y7,Q9Y5Y7,NaN,LYVE1,NaN,26639.6,37457.2,49709.4,30120.5,36232.6,...,66596.0,70511.9,68331.6,59306.4,48793.4,107999.0,68237.1,44161.4,NaN,NaN
677,Q9Y646,Q9Y646,NaN,CPQ,NaN,12229.7,9045.77,9757.3,14903.9,13347.3,...,9094.25,9484.76,15927.0,10066.5,14565.6,12017.5,15823.9,20288.9,NaN,NaN
678,Q9Y6N7,Q9Y6N7,NaN,ROBO1,NaN,4208.16,2697.48,4508.26,5846.31,4531.03,...,4135.06,9523.76,NaN,4705.79,7478.49,2069.04,3888.02,NaN,NaN,NaN


# Filter rows based on categorical column

Getting rid of any rows that have a "+" value for the "only identified by site" column. This won't work for DIA as this column does not exist, but will work for DDA.

In [16]:
df

,Majority protein IDs,Protein.Ids,Protein.Names,Genes,First.Protein.Description,Intensity 85,Intensity QC_G10,Intensity QC_A3,Intensity 75,Intensity 71,...,Intensity 127,Intensity 31,Intensity 1,Intensity 115,Intensity 35,Intensity 91,Intensity 121,Intensity 107,Potential contaminant,Reverse
Label,,,,,,Neurotypical,QC,QC,Neurotypical,Autism,...,Neurotypical,Neurotypical,Autism,Neurotypical,Autism,Neurotypical,Neurotypical,Autism,,
0,A0A075B6H7,A0A075B6H7,NaN,IGKV3-7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,780180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0A075B6K4,A0A075B6K4,NaN,IGLV3-10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,306602.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A0A075B6S2,A0A075B6S2,NaN,IGKV2D-29,NaN,1471.09,NaN,NaN,NaN,NaN,...,NaN,NaN,25570.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A0A075B6S5,A0A075B6S5,NaN,IGKV1-27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,36935.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,Q9Y5C1,Q9Y5C1,NaN,ANGPTL3,NaN,5577.84,4138.06,4011.38,5248.27,3342.73,...,4269.59,3281.33,NaN,4102.91,2963.73,3964.51,2533.34,NaN,NaN,NaN
676,Q9Y5Y7,Q9Y5Y7,NaN,LYVE1,NaN,26639.6,37457.2,49709.4,30120.5,36232.6,...,66596.0,70511.9,68331.6,59306.4,48793.4,107999.0,68237.1,44161.4,NaN,NaN
677,Q9Y646,Q9Y646,NaN,CPQ,NaN,12229.7,9045.77,9757.3,14903.9,13347.3,...,9094.25,9484.76,15927.0,10066.5,14565.6,12017.5,15823.9,20288.9,NaN,NaN
678,Q9Y6N7,Q9Y6N7,NaN,ROBO1,NaN,4208.16,2697.48,4508.26,5846.31,4531.03,...,4135.06,9523.76,NaN,4705.79,7478.49,2069.04,3888.02,NaN,NaN,NaN


In [18]:
df = df[df["Only identified by site"] != "+"]
df

KeyError: 'Only identified by site'

Getting rid of any rows that have a non NaN value for the "Reverse" column.

In [19]:
df = df[df["Reverse"] != "+"]
df

,Majority protein IDs,Protein.Ids,Protein.Names,Genes,First.Protein.Description,Intensity 85,Intensity QC_G10,Intensity QC_A3,Intensity 75,Intensity 71,...,Intensity 127,Intensity 31,Intensity 1,Intensity 115,Intensity 35,Intensity 91,Intensity 121,Intensity 107,Potential contaminant,Reverse
Label,,,,,,Neurotypical,QC,QC,Neurotypical,Autism,...,Neurotypical,Neurotypical,Autism,Neurotypical,Autism,Neurotypical,Neurotypical,Autism,,
0,A0A075B6H7,A0A075B6H7,NaN,IGKV3-7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,780180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0A075B6K4,A0A075B6K4,NaN,IGLV3-10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,306602.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A0A075B6S2,A0A075B6S2,NaN,IGKV2D-29,NaN,1471.09,NaN,NaN,NaN,NaN,...,NaN,NaN,25570.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A0A075B6S5,A0A075B6S5,NaN,IGKV1-27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,36935.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,Q9Y5C1,Q9Y5C1,NaN,ANGPTL3,NaN,5577.84,4138.06,4011.38,5248.27,3342.73,...,4269.59,3281.33,NaN,4102.91,2963.73,3964.51,2533.34,NaN,NaN,NaN
676,Q9Y5Y7,Q9Y5Y7,NaN,LYVE1,NaN,26639.6,37457.2,49709.4,30120.5,36232.6,...,66596.0,70511.9,68331.6,59306.4,48793.4,107999.0,68237.1,44161.4,NaN,NaN
677,Q9Y646,Q9Y646,NaN,CPQ,NaN,12229.7,9045.77,9757.3,14903.9,13347.3,...,9094.25,9484.76,15927.0,10066.5,14565.6,12017.5,15823.9,20288.9,NaN,NaN
678,Q9Y6N7,Q9Y6N7,NaN,ROBO1,NaN,4208.16,2697.48,4508.26,5846.31,4531.03,...,4135.06,9523.76,NaN,4705.79,7478.49,2069.04,3888.02,NaN,NaN,NaN


Getting rid of any rows that have a non NaN value for the "Potential contaminant" column. 

In [20]:
newDF = df[df["Potential contaminant"] != "+"]
newDF

,Majority protein IDs,Protein.Ids,Protein.Names,Genes,First.Protein.Description,Intensity 85,Intensity QC_G10,Intensity QC_A3,Intensity 75,Intensity 71,...,Intensity 127,Intensity 31,Intensity 1,Intensity 115,Intensity 35,Intensity 91,Intensity 121,Intensity 107,Potential contaminant,Reverse
Label,,,,,,Neurotypical,QC,QC,Neurotypical,Autism,...,Neurotypical,Neurotypical,Autism,Neurotypical,Autism,Neurotypical,Neurotypical,Autism,,
0,A0A075B6H7,A0A075B6H7,NaN,IGKV3-7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,780180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0A075B6K4,A0A075B6K4,NaN,IGLV3-10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,306602.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A0A075B6S2,A0A075B6S2,NaN,IGKV2D-29,NaN,1471.09,NaN,NaN,NaN,NaN,...,NaN,NaN,25570.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A0A075B6S5,A0A075B6S5,NaN,IGKV1-27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,36935.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,Q9Y5C1,Q9Y5C1,NaN,ANGPTL3,NaN,5577.84,4138.06,4011.38,5248.27,3342.73,...,4269.59,3281.33,NaN,4102.91,2963.73,3964.51,2533.34,NaN,NaN,NaN
676,Q9Y5Y7,Q9Y5Y7,NaN,LYVE1,NaN,26639.6,37457.2,49709.4,30120.5,36232.6,...,66596.0,70511.9,68331.6,59306.4,48793.4,107999.0,68237.1,44161.4,NaN,NaN
677,Q9Y646,Q9Y646,NaN,CPQ,NaN,12229.7,9045.77,9757.3,14903.9,13347.3,...,9094.25,9484.76,15927.0,10066.5,14565.6,12017.5,15823.9,20288.9,NaN,NaN
678,Q9Y6N7,Q9Y6N7,NaN,ROBO1,NaN,4208.16,2697.48,4508.26,5846.31,4531.03,...,4135.06,9523.76,NaN,4705.79,7478.49,2069.04,3888.02,NaN,NaN,NaN


# Filtering the Numerical Data

Focusing on the filtering and transformation of Numerical Data 

In [22]:
newDF = df.transpose()
newDF

,Label,0,1,2,3,4,5,6,7,8,...,670,671,672,673,674,675,676,677,678,679
Majority protein IDs,,A0A075B6H7,A0A075B6K4,A0A075B6S2,A0A075B6S5,A0A075B7D0,A0A087WSY6,A0A0A0MS15,A0A0B4J1Y8,A0A0B4J1Y9,...,Q9UQ80,Q9Y287,Q9Y2G1,Q9Y490,Q9Y4L1,Q9Y5C1,Q9Y5Y7,Q9Y646,Q9Y6N7,Q9Y6Z7
Protein.Ids,,A0A075B6H7,A0A075B6K4,A0A075B6S2,A0A075B6S5,A0A075B7D0,A0A087WSY6,A0A0A0MS15,A0A0B4J1Y8,A0A0B4J1Y9,...,Q9UQ80,Q9Y287,Q9Y2G1,Q9Y490,Q9Y4L1,Q9Y5C1,Q9Y5Y7,Q9Y646,Q9Y6N7,Q9Y6Z7
Protein.Names,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Genes,,IGKV3-7,IGLV3-10,IGKV2D-29,IGKV1-27,IGHV1OR15-1,IGKV3D-15,IGHV3-49,IGLV9-49,IGHV3-72,...,PA2G4,ITM2B,MYRF,TLN1,HYOU1,ANGPTL3,LYVE1,CPQ,ROBO1,COLEC10
First.Protein.Description,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Intensity 91,Neurotypical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5793.53,4894.96,7970.31,12984.0,11005.1,3964.51,107999.0,12017.5,2069.04,24553.1
Intensity 121,Neurotypical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,14452.4,19011.5,10740.1,2533.34,68237.1,15823.9,3888.02,28674.6
Intensity 107,Autism,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,25034.9,10229.5,NaN,44161.4,20288.9,NaN,22333.6
Potential contaminant,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Creating multiple dataframes for each grouping makes it easier to filter by the valid values in these groups.

In [23]:
group1 = newDF[newDF["Label"] == "Autism"]
group1 = group1.drop("Label", axis = 1)
group2 = newDF[newDF["Label"] == "Neurotypical"]
group2 = group2.drop("Label", axis = 1)
group3 = newDF[newDF["Label"] == "QC"]
group3 = group3.drop("Label", axis = 1)

valid_values_filter is a function that takes a dataframe as a parameter (df). It creates a list of valid columns in this dataframe based on whether this column has 70% or above valid (>= 1) values. 

In [24]:
def valid_values_filter(df):
    valid_cols = []
    for col in df:
        valid_values = 0
        for val in df[col].values:
            if val >= 1:
                valid_values +=1
        if valid_values/len(df[col].values) >= 0.7:
            valid_cols.append(col)
    return(valid_cols)

group1_valid_vals = valid_values_filter(group1)
group2_valid_vals = valid_values_filter(group2)
group3_valid_vals = valid_values_filter(group3)

The list of groups can be combined into a set and then have the "Labels" column added in so that the dataframe is filtered taking only these column values into account.

In [25]:
valid_vals = group1_valid_vals + group2_valid_vals + group3_valid_vals
valid_vals_set = sorted(set(valid_vals))
valid_list = ["Label"] + valid_vals_set
newDF_filtered = newDF[valid_list]
newDF_filtered

,Label,13,15,18,19,20,21,22,23,24,...,670,671,672,673,674,675,676,677,678,679
Majority protein IDs,,A1L4H1,A6QL64,O00151,O00187,O00299,O00391,O00468,O00533,O00592,...,Q9UQ80,Q9Y287,Q9Y2G1,Q9Y490,Q9Y4L1,Q9Y5C1,Q9Y5Y7,Q9Y646,Q9Y6N7,Q9Y6Z7
Protein.Ids,,A1L4H1,A6QL64,O00151,O00187,O00299,O00391,O00468,O00533,O00592,...,Q9UQ80,Q9Y287,Q9Y2G1,Q9Y490,Q9Y4L1,Q9Y5C1,Q9Y5Y7,Q9Y646,Q9Y6N7,Q9Y6Z7
Protein.Names,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Genes,,SSC5D,ANKRD36,PDLIM1,MASP2,CLIC1,QSOX1,AGRN,CHL1,PODXL,...,PA2G4,ITM2B,MYRF,TLN1,HYOU1,ANGPTL3,LYVE1,CPQ,ROBO1,COLEC10
First.Protein.Description,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Intensity 91,Neurotypical,9403.47,60601.3,NaN,120969.0,3971.27,40977.6,3549.6,37416.7,6317.68,...,5793.53,4894.96,7970.31,12984.0,11005.1,3964.51,107999.0,12017.5,2069.04,24553.1
Intensity 121,Neurotypical,8095.36,74928.5,NaN,189209.0,3953.04,40649.3,2966.7,50585.2,13302.9,...,NaN,NaN,14452.4,19011.5,10740.1,2533.34,68237.1,15823.9,3888.02,28674.6
Intensity 107,Autism,4286.93,9823.51,NaN,83563.2,NaN,36676.0,NaN,43330.6,12113.5,...,NaN,NaN,NaN,25034.9,10229.5,NaN,44161.4,20288.9,NaN,22333.6
Potential contaminant,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
newDF_filtered = newDF_filtered.transpose()

Export the dataframe to be used as an input file for the R analysis.

In [27]:
newDF_filtered.to_csv(r'C:\Users\ASUS\Desktop\export_DIA_table.csv', index = False, header = True)